# Net surgery to produce a `Graph` model for regressing poselets

In [ ]:
%matplotlib inline

from keras.optimizers import SGD

import numpy as np

import models
from vggnet.upgrade_weights import upgrade_weights
from vggnet.vgg16_keras import VGG_16

In [ ]:
models = reload(models)
solver = SGD()
rgb_shape = (6, 224, 224)
flow_shape = (2, 224, 224)
regressor_outputs = 6
init = 'glorot_normal'

In [ ]:
huge = models.vggnet16_poselet_class_flow({
    'images': (6, 224, 224),
    'flow': (2, 224, 224),
    'poselet': (301,)
}, solver, init)

In [ ]:
flow_seq = huge.nodes['flow_conv']
rgb_seq = huge.nodes['rgb_conv']
ilsvrc_weights_path = './vggnet/vgg16_weights.h5'
ilsvrc_model = VGG_16(ilsvrc_weights_path)
upgrade_weights(flow_seq.layers, ilsvrc_model.layers)
upgrade_weights(rgb_seq.layers, ilsvrc_model.layers)

In [ ]:
front_layers = len(flow_seq.layers)
assert front_layers == len(rgb_seq.layers), "Flow and RGB pipelines should be same length"
back_ilsvrc_layers = ilsvrc_model.layers[front_layers:]
back_seq = huge.nodes['shared_layers']
upgrade_weights(back_seq.layers, back_ilsvrc_layers)

In [ ]:
huge.save_weights('vggnet/vgg16-2stream-pslt-clas.h5')